<a href="https://colab.research.google.com/github/kalebsampaco/Ejercicios-en-google-colab/blob/master/Redes_de_decodificaci%C3%B3n_codificaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import *
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
from keras.callbacks import LearningRateScheduler, ModelCheckpoint


#CARGADO DE IMAGENES

path_entradas = '/content/sample_data/2d_images'
path_salidas = '/content/sample_data/2d_masks'
alto, ancho = 32, 32

imagenes = [x for x in sorted(os.listdir(path_entradas)) if x[-4:] == '.tif']

entradas = np.empty((len(imagenes), alto, ancho), dtype='float32')
for i, nombre in enumerate(imagenes):
    imagen = cv2.imread(path_entradas + nombre, cv2.IMREAD_UNCHANGED).astype('int16').astype('float32')
    imagen = cv2.resize(imagen, dsize=(ancho, alto), interpolation=cv2.INTER_CUBIC)
    imagen = (imagen - np.min(imagen)) / (np.max(imagen) - np.min(imagen))
    entradas[i] = imagen

salidas = np.empty((len(imagenes), alto, ancho), dtype='float32')
for i, nombre in enumerate(imagenes):
    imagen = cv2.imread(path_salidas + nombre, cv2.IMREAD_UNCHANGED).astype('float32')/255.
    imagen = cv2.resize(imagen, dsize=(ancho, alto), interpolation=cv2.INTER_NEAREST)
    salidas[i] = imagen

fig, ax = plt.subplots(1,2)
ax[0].imshow(entradas[0], cmap='gray')
ax[1].imshow(salidas[0], cmap='gray')
plt.show()

# ADAPTACION Y DIVISION DE SETS DE TEST Y ENTRENAMIENTO
entradas = np.expand_dims(entradas,axis=3)
salidas = np.expand_dims(salidas,axis=3)
entradas_train, entradas_test, salidas_train, salidas_test = train_test_split(entradas, salidas, test_size = 0.2)


#DEFINICION Y ENTRENAMIENTO DE MODELO

entrada = Input(shape=entradas_train.shape[1:])
x = Conv2D(filters=8, kernel_size=(3,3), activation='relu', padding='valid')(entrada)
x = MaxPool2D(strides=(2,2))(x)
x = Conv2D(filters=16, kernel_size=(3,3), activation='relu', padding='valid')(x)
x = Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='valid')(x)

x = Conv2DTranspose(filters=32, kernel_size=(3,3), activation='relu', padding='valid')(x)
x = Conv2DTranspose(filters=16, kernel_size=(3,3), activation='relu', padding='valid')(x)
x=UpSampling2D(2)(x)
x = Conv2DTranspose(filters=8, kernel_size=(3,3), activation='relu',padding='valid')(x)
salida = Conv2D(filters=1, kernel_size=(3,3), activation='sigmoid',padding='same')(x)
                                                         
modelo = Model(entrada,salida)
modelo.summary()

def generador_datos(entradas_train, salidas_train, batch):
    generador_entradas = ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,rotation_range=10,zoom_range=0.1).flow(entradas_train, entradas_train, batch,seed=42)
    generador_salidas = ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,rotation_range=10,zoom_range=0.1).flow(salidas_train, salidas_train, batch,seed=42)
    while True:
        entradas_batch, _ = generador_entradas.next()
        salidas_batch, _ = generador_salidas.next()
        yield entradas_batch, salidas_batch


entradas_batch, salidas_batch = next(generador_datos(entradas_train, salidas_train, 8))

modelo.compile(optimizer=Adam(2e-4), loss='binary_crossentropy', metrics=['accuracy'])

hist = modelo.fit_generator(generador_datos(entradas_train, salidas_train, 8),steps_per_epoch = 200,validation_data = (entradas_test, salidas_test),epochs=10, verbose=2)

#EVALUACION DE MODELO
plt.figure(1)
plt.plot(hist.history['loss'], color='b')
plt.plot(hist.history['val_loss'], color='r')
plt.figure(2)
plt.plot(hist.history['acc'], color='b')
plt.plot(hist.history['val_acc'], color='r')
plt.show()

prediccion = modelo.predict(entradas_test)
fig, ax = plt.subplots(1,3,figsize=(12,6))
ax[0].imshow(entradas_test[0,:,:,0], cmap='gray')
ax[1].imshow(salidas_test[0,:,:,0])
ax[2].imshow(prediccion[0,:,:,0])
plt.show()


AttributeError: ignored